In [1]:
from bio_embeddings.embed import ProtTransBertBFDEmbedder#
## bigger embeddings : https://docs.bioembeddings.com/v0.2.1/api/bio_embeddings.embed.html#bio_embeddings.embed.ProtTransT5XLU50Embedder
from bio_embeddings.embed import ProtTransT5XLU50Embedder, ESM1bEmbedder# , ProtTransT5UniRef50Embedder,ProtTransT5BFDEmbedder, ProtTransT5XLU50Embedder
from Bio import SeqIO

import pandas as pd
import numpy as np
import random

# from tqdm import tqdm
from tqdm.autonotebook import tqdm

In [2]:
# pip install -U "bio-embeddings[all] @ git+https://github.com/sacdallago/bio_embeddings.git"


In [3]:
##ORIG
# !wget https://github.com/ddofer/proteinA/raw/main/SWP_human_viruses_all.fasta --output-document swp_human_vir_all.fasta

## TEMP -local:
df = pd.read_csv("df_meta_v2.csv.gz")

df

C:\apps\anaconda3\envs\bio\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Entry,Gene names (primary ),Entry name,Protein names,Organism,Length,Sequence,Mass,Keywords,Features,...,Involvement in disease,Virus hosts,Catalytic activity,Tissue specificity,Developmental stage,Induction,Peptide,Propeptide,Temperature dependence,Interacts with
0,Q8V0N6,NaN,POLG_HAV88,Genome polyprotein [Cleaved into: Capsid prote...,Human hepatitis A virus genotype IIB (isolate ...,2227,MNMSRQGIFQTVGSGLDHILSLADIEEEQMIQSVDRTAVTGASYFT...,"251,728",ATP-binding;Capsid protein;Coiled coil;Covalen...,Active site (4); Chain (19); Coiled coil (1); ...,...,NaN,Homo sapiens (Human) [TaxID: 9606],CATALYTIC ACTIVITY: Reaction=a ribonucleoside ...,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Q5Y944,NaN,POLG_HAVCF,Genome polyprotein [Cleaved into: Capsid prote...,Human hepatitis A virus genotype IIA (isolate ...,2225,MNMSRQGIFQTVGSGLDHILSLADIEEEQMIQSVDRTAVTGASYFT...,"251,415",ATP-binding;Capsid protein;Coiled coil;Covalen...,Active site (3); Chain (19); Coiled coil (1); ...,...,NaN,Homo sapiens (Human) [TaxID: 9606],CATALYTIC ACTIVITY: Reaction=a ribonucleoside ...,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Q67825,NaN,POLG_HAVGB,Genome polyprotein [Cleaved into: Capsid prote...,Human hepatitis A virus genotype IA (isolate G...,2227,MNMSKQGIFQTVGSGLDHILSLADIEEEQMIQSVDRTAVTGASYFT...,"251,565",ATP-binding;Capsid protein;Coiled coil;Covalen...,Active site (3); Chain (19); Coiled coil (1); ...,...,NaN,Homo sapiens (Human) [TaxID: 9606],CATALYTIC ACTIVITY: Reaction=a ribonucleoside ...,NaN,NaN,NaN,NaN,NaN,NaN,0
3,P08617,NaN,POLG_HAVHM,Genome polyprotein [Cleaved into: Capsid prote...,Human hepatitis A virus genotype IB (isolate H...,2227,MNMSRQGIFQTVGSGLDHILSLADIEEEQMIQSVDRTAVTGASYFT...,"251,508",3D-structure;ATP-binding;Capsid protein;Covale...,Active site (3); Beta strand (66); Chain (19);...,...,NaN,Cercopithecus hamlyni (Owl-faced monkey) (Haml...,CATALYTIC ACTIVITY: Reaction=a ribonucleoside ...,NaN,NaN,NaN,NaN,NaN,NaN,0
4,A5LGW7,NaN,POLG_HAVJ8,Genome polyprotein [Cleaved into: Capsid prote...,Human hepatitis A virus genotype IIIB (isolate...,2228,MNMSRQGIFQTVGSGLDHILSLADVEEEQMIQSVDRTAVTGASYFT...,"251,802",ATP-binding;Capsid protein;Coiled coil;Covalen...,Active site (3); Chain (19); Coiled coil (1); ...,...,NaN,Cercopithecus hamlyni (Owl-faced monkey) (Haml...,CATALYTIC ACTIVITY: Reaction=a ribonucleoside ...,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36899,Q9P2T1,GMPR2,GMPR2_HUMAN,GMP reductase 2 (GMPR 2) (EC 1.7.1.7) (Guanosi...,Homo sapiens (Human),348,MPHIDNDVKLDFKDVLLRPKRSTLKSRSEVDLTRSFSFRNSKQTYS...,"37,874",3D-structure;Acetylation;Alternative splicing;...,Active site (2); Alternative sequence (2); Bet...,...,NaN,NaN,CATALYTIC ACTIVITY: Reaction=IMP + NADP(+) + N...,"TISSUE SPECIFICITY: Highly expressed in heart,...",NaN,NaN,NaN,NaN,NaN,1
36900,Q8IUX8,EGFL6,EGFL6_HUMAN,Epidermal growth factor-like protein 6 (EGF-li...,Homo sapiens (Human),553,MPLPWSLALPLLLSWVAGGFGNAASARHHGLLASARQPGVCHYGTK...,"61,317",Alternative splicing;Basement membrane;Calcium...,Alternative sequence (1); Chain (1); Coiled co...,...,NaN,NaN,NaN,NaN,DEVELOPMENTAL STAGE: Only detected in fetal ti...,NaN,NaN,NaN,NaN,1
36901,O15372,EIF3H,EIF3H_HUMAN,Eukaryotic translation initiation factor 3 sub...,Homo sapiens (Human),352,MASRKEGTGSTATSSSSTAGAAGKGKGKGGSGDSAVKQVQIDGLVV...,"39,930",3D-structure;Cytoplasm;Initiation factor;Isope...,Beta strand (7); Chain (1); Compositional bias...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
36902,P54852,EMP3,EMP3_HUMAN,Epithelial membrane protein 3 (EMP-3) (Hematop...,Homo sapiens (Human),163,MSLLLLVVSALHILILILLFVATLDKSWWTLPGKESLNLWYDCTWN...,"18,429",Glycoprotein;Membrane;Reference proteome;Trans...,Chain (1); Glycosylation (2); Natural variant ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49


In [4]:
## could download and gunzip models manually 
# !wget http://data.bioembeddings.com/public/embeddings/embedding_models/ XXXXX
# !gunzip 

In [5]:
MY_FASTA_PATH ="swp_human_vir_all.fasta"

# Embed sequences in a FASTA file

In [6]:
sequences = df["Sequence"].str[0:1022].values
sequences

array(['MNMSRQGIFQTVGSGLDHILSLADIEEEQMIQSVDRTAVTGASYFTSVDQSSVHTAEVGSHQVEPLKTSVDKPGSKKTQGEKFFLIHSADWLTTHALFHEVAKLDVVKLLYNEQFAVQGLLRYHTYARFGIEIQVQINPTPFQQGGLICAMVPGDQSYGSIASLTVYPHGLLNCNINNVVRIKVPFIYTRGAYHFKDPQYPVWELTIRVWSELNIGTGTSAYTSLNVLARFTDLELHGLTPLSTQMMRNEFRVSTTENVVNLSNYEDARAKMSFALDQEDWKSDPSQGGRIKITHFTTWTSIPTLAAQFPFNASDSVGQQIKVIPVDPYFFQMTNSNPDQKCITALASICQMFCFWRGDLVFDFQVFPTKYHSGRLLFCFVPGNELIDVSGITLKQATTAPCAVMDITGVQSTLRFRVPWISDTPYRVNRYTKSAHQKGEYTAIGKLIVYCYNRLTSPSNVASHVRVNVYLSAINLECFAPLYHAMDVTTQVGDDSGGFSTTVSTEQNVPDPQVGITTMRDLKGRANRGKMDVSGVQAPVGAITTIEDPVLAKKVPETFPELKPGESRHTSDHMSIYKFMGRSHFLCTFTFNSNNKEYTFPITLSSTSNPPHGLPSTLRWFFNLFQLYRGPLDLTIIITGATDVDGMAWFTPVGLAVDTPWVEKESALSIDYKTALGAVRFNTRRTGNIQIRLPWYSYLYAVSGALDGLGDKTDSTFGLVSIQIANYNHSDEYLSFSCYLSVTEQSEFYFPRAPLNSNAMLSTETMMSRIAAGDLESSVDDPRSEEDRRFESHIESRKPYKELRLEVGKQRLKYAQEELSNEVLPPPRKMKGLFSQAKISLFYTEDHEIMKFSWRGVTADTRALRRFGFSLAAGRSVWTLEMDAGVLTGRLVRLNDEKWTEMKDDKIVSLVEKFTSNKHWSKVNFPHGMLDLEEIAANSKDFPNMSETDLCFLLHWLNPKKINLADRMLGLSGVQEIKEQGIGLIAECRTFL

In [7]:
# sequences = []
# # for record in SeqIO.parse("tiny_sampled.fasta", "fasta"): # ORIG from demo - tiny file
# for record in SeqIO.parse(MY_FASTA_PATH, "fasta"): # run on all our data
#     sequences.append(record)

In [8]:
sequences[1]

'MNMSRQGIFQTVGSGLDHILSLADIEEEQMIQSVDRTAVTGASYFTSVDQSSVHTAEVGSHQIEPLKTSVDKPGSKKTQGEKFFLIHSADWLTTHALFHEVAKLDVVKLLYNEQFAVQGLLRYHTYARFGIEIQVQINPTPFQQGGLICAMVPGDQSYGSIASLTVYPHGLLNCNINNVVRIKVPFIYTRGAYHFKDPQYPVWELTIRVWSELNIGTGTSAYTSLNVLARFTDLELHGLTPLSTQMMRNEFRVSTTENVVNLSNYEDARAKMSFALDQEDWKSDPSQGGGIKITHFTTWTSIPTLAAQFPFNASDSVGQQIKVIPVDPYFFQMTNSNPDQKCITALASICQMFCFWRGDLVFDFQVFPTKYHSGRLLFCFVPGNELIDVSGITLKQATTAPCAVMDIAGVQSTLRFRVPWIPDTRYRVNRYTKSAHQKGEYTAIGKLIVYCYNRLTSPSNVASHVRVNVYLSAINLECFAPLYHAMDVTTQVGDDSGGFSTTVSTEQNVPDPQVGITTMRDLKGKANRGKMDVSGVQAPVGAITTIEDPVLAKKVPETFPELKPGESRHTSDHMSIYKFMGRSHFLCTFTFNSNNKEYTFPITLSSTSNPPHGLPSTLRWFFNLFQLYRGPLDLTIIITGATDVDGMAWFTPVGLAVDTPWVEKESALSIDYKTALGAVRFNTRRTGNIQIRLPWYSYLYAVSGALDGLGDKTDSTFGLVSIQIANYNHSDEYLSFSCYLSVTEQSEFYFPRAPLNSNAMLSTETMMSRIAAGDLESSVDDPRSEEDRRFESHIESRKPYKELRLEVGKQRLKYAQEELSNEVLPPPRKMKGLFSQAKISLFYTEEHEIMKFSWRGVTADTRALRRFGFSLAAGRSVWTLEMDAGVLTGRLVRLNDEKWTEMKDDKIVSLVEKFTSNKHWSKVNFPHGMLDLEEIAANSKDFPNMSETDLCFLLHWLNPKKINLADRMLGLSGVQEIKEQGIGLIAECRTFLDSITGSL

In [9]:
# # sequences ## seqrecord item
# print(len(sequences))
# print(len([s for s in sequences if "X" not in str(s.seq)]),"# sequences without unknown AA")

# # # ## drop sequences with unknown AA / X  - OPTIONAL
# # sequences = [s for s in sequences if "X" not in str(s.seq)]

# # sequences = sequences[0:3000] # sample  data

# print(len(sequences))

In [10]:
# embedder = ProtTransBertBFDEmbedder(half_model=True,half_precision_model=True)
# embedder =ProtTransT5XLU50Embedder(half_model=True,half_precision_model=True) ## DownLoad error with all?
# embedder =ProtTransT5BFDEmbedder(half_model=True)
embedder =ESM1bEmbedder(half_model=True,half_precision_model=True)

# embedder = SeqVecEmbedder(half_model=True) ## slightly smaller, faster

RuntimeError: unexpected EOF, expected 1590334 more bytes. The file might be corrupted.

In [ ]:
%%time
# ## ORIG
# embeddings = embedder.embed_many([str(s.seq) for s in sequences]) # ORIG

# embeddings = embedder.embed_many([str(s.seq)[0:256] for s in sequences]) # Workaround for long sequences

# # `embed_many` returns a generator.
# # We want to keep both RAW embeddings and reduced embeddings in memory.
# # To do so, we simply turn the generator into a list!
# # (this will start embedding the sequences!)

# # embeddings = list(embeddings) ## embeddings per position

# reduced_embeddings = [ProtTransBertBFDEmbedder.reduce_per_protein(e) for e in embeddings] ## embeddings over whole sequence

In [ ]:
%%time
## 12 min for 4k rows, max 300 length
# ### new/ALT - get reduced embeddings only, without batch? # - requires embed first
# reduced_embeddings = [ProtTransBertBFDEmbedder.reduce_per_protein(s) for str(s.seq)[0:256] in sequences]

embeddings = embedder.embed_many([str(s.seq)[0:1020] for s in sequences]) ## embeddings per position # truncated for Workaround for long sequences

In [ ]:
%%time
#### total: 1h 14min 41s
# ## try maybe embedder insyead of ProtTransBertBFDEmbedder ??
# reduced_embeddings = [ProtTransBertBFDEmbedder.reduce_per_protein(e) for e in embeddings]  ## embeddings over whole sequence # ORI

### ALT : [embedder.reduce_per_protein(embedding) for embedding in embedder.embed_many(data['HEAVY CDR3 (aa)'])]
# reduced_embeddings = [embedder.reduce_per_protein(e) for e in tqdm.tqdm(embeddings)]  ## embeddings over whole sequence # ORI

reduced_embeddings = [embedder.reduce_per_protein(e) for e in tqdm(embeddings)]  ## embeddings over whole sequence # ORI


In [ ]:
# for (per_amino_acid, per_protein) in zip(embeddings[0:19], reduced_embeddings[0:19]):
#     print(per_amino_acid.shape, per_protein.shape)


In [ ]:
print(len(reduced_embeddings))
print(reduced_embeddings[0].shape)

In [ ]:
reduced_embeddings[0]

In [ ]:
names = [str(s.name) for s in sequences]
print(len(names))
names[0:2]

In [ ]:
ids = [str(s.id) for s in sequences]
print(len(ids))
ids[0:2]

In [ ]:
descriptions = [str(s.description).replace(" <unknown description>","") for s in sequences]
print(len(descriptions))
descriptions[0:2]

In [ ]:
df_meta = pd.DataFrame(data={"ID":ids,"name":names,"descriptions":descriptions})
# df_meta["ID"] = df_meta["ID"].str.split("_",expand=True,n=1)[1].str.strip() ## get uniprot id, instead of uniref90_+uniprot ID

## add some text features
df_meta["fragment"] = (df_meta["descriptions"].str.contains("Fragment",na=False)).astype("int")
df_meta["Unknown"] = (df_meta["descriptions"].str.contains("Uncharacterized|Unknown",case=False)).astype("int")
df_meta["word_count"] = df_meta["descriptions"].str.split().str.len()
df_meta["length"] = [len(str(s.seq)) for s in sequences]
print(df_meta.describe().round(2))
# df_meta

In [ ]:
df = pd.DataFrame(data=reduced_embeddings) # numpy to pandas
 ## lengths of all the seqs
# df

In [ ]:
assert df_meta.shape[0]==df.shape[0]

In [ ]:
df = pd.concat([df_meta,df],axis=1)
df

#### Save output as parquet file to disc/drive (local runtime)
* Can also save as csv if wanted. 

In [ ]:
df.columns = df.columns.astype(str)
df.to_parquet("swp_human_viri_all_embed_esm.parquet")

In [ ]:
# df.to_csv("swp_human_viri_all_embed.csv.gz",index=False,compression="gzip")